In [1]:
# importando as bibliotecas necessárias

import numpy as np                         # biblioteca para cálculos matemáticos
import pandas as pd                        # biblioteca para manipulação de dados 
import torch                               # biblioteca para cálculos com GPUs
import torch.nn as nn                      # biblioteca para criação de redes neurais 
import torch.nn.parallel                   # biblioteca para criação de redes neurais paralelas
import torch.optim as optim                # biblioteca para otimização de redes neurais
from torch.autograd import Variable        # biblioteca para criação de variáveis
import torch.utils.data                    # biblioteca para criação de dados

In [2]:
# carregando os dados

# filmes
movies = pd.read_csv('./data/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')

# usuários
users = pd.read_csv('./data/users.dat', sep='::', header=None, engine='python', encoding='latin-1')

# avaliações dos usuários
ratings = pd.read_csv('./data/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [4]:
# visualizando os primeiros registros dos filmes

movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
# visualizando os primeiros registros dos usuários

users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [6]:
# visualizando os primeiros registros das avaliações

ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [9]:
# carregando os dados de treino e teste

training_set = pd.read_csv('./data/u1.base', sep='\t')
test_set = pd.read_csv('./data/u1.test', sep='\t')

In [10]:
# transformando os dados em um array numpy

training_set = np.array(training_set, dtype='int')
test_set = np.array(test_set, dtype='int')

In [11]:
# pegando o numero máximo de usuários e filmes

nb_users = int(max(max(training_set[ :, 0]), max(test_set[ :, 0])))
nb_movies = int(max(max(training_set[ :, 1]), max(test_set[ :, 1])))

In [12]:
# função para transformação dos dados de treino e teste para uma lista(users) de listas(movies)

def convert(data):
    new_data = []
    
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    
    return new_data

In [13]:
# transformando os dados de treino e teste para uma lista(users) de listas(movies)

training_set = convert(training_set)
test_set = convert(test_set)

In [14]:
# convertendo os dados de treino e teste para o formato torch

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [18]:
class SAE(nn.Module):

    def __init__(self, ):
        super(SAE, self).__init__()           # chamando o construtor da classe mãe
        
        # definindo parametros da camada de entrada
        self.fc1 = nn.Linear(nb_movies, 20)    # 20, numeros de features da camada de entrada
        self.fc2 = nn.Linear(20, 10)           # 10, numeros de features da camada oculta
        self.fc3 = nn.Linear(10, 20)           # 20, numeros de features da camada de saída
        self.fc4 = nn.Linear(20, nb_movies)    # nb_movies, numeros de features da camada de saída
        self.activation = nn.Sigmoid()         # função de ativação

    def forward(self, x):
        
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        
        return x

In [19]:
# instanciando a classe SAE

sae = SAE()

In [21]:
# parametros da rede

criteon = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)
nb_epochs = 200

In [22]:
# treinando a rede

for epoch in range(1, nb_epochs +1):
    
    train_loss = 0
    s = 0.

    for id_user in range(nb_users):
        
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()

        if torch.sum(target.data > 0) > 0:
        
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criteon(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()

    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss/s))

epoch: 1 loss: tensor(1.7711)
epoch: 2 loss: tensor(1.0967)
epoch: 3 loss: tensor(1.0535)
epoch: 4 loss: tensor(1.0382)
epoch: 5 loss: tensor(1.0311)
epoch: 6 loss: tensor(1.0264)
epoch: 7 loss: tensor(1.0236)
epoch: 8 loss: tensor(1.0220)
epoch: 9 loss: tensor(1.0205)
epoch: 10 loss: tensor(1.0197)
epoch: 11 loss: tensor(1.0188)
epoch: 12 loss: tensor(1.0186)
epoch: 13 loss: tensor(1.0178)
epoch: 14 loss: tensor(1.0177)
epoch: 15 loss: tensor(1.0172)
epoch: 16 loss: tensor(1.0171)
epoch: 17 loss: tensor(1.0168)
epoch: 18 loss: tensor(1.0164)
epoch: 19 loss: tensor(1.0163)
epoch: 20 loss: tensor(1.0163)
epoch: 21 loss: tensor(1.0161)
epoch: 22 loss: tensor(1.0162)
epoch: 23 loss: tensor(1.0161)
epoch: 24 loss: tensor(1.0159)
epoch: 25 loss: tensor(1.0159)
epoch: 26 loss: tensor(1.0157)
epoch: 27 loss: tensor(1.0154)
epoch: 28 loss: tensor(1.0150)
epoch: 29 loss: tensor(1.0134)
epoch: 30 loss: tensor(1.0112)
epoch: 31 loss: tensor(1.0100)
epoch: 32 loss: tensor(1.0076)
epoch: 33 loss: t

In [23]:
# testando o modelo

test_loss = 0
s = 0.

for id_users in range(nb_users):
    
    input = Variable(training_set[id_users]).unsqueeze(0)
    target = Variable(test_set[id_users]).unsqueeze(0)

    if torch.sum(target.data > 0) > 0:
        
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criteon(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.

print('test loss: ' + str(test_loss/s))

test loss: tensor(0.9507)
